In [1]:
import pandas as pd
import numpy as np
import dtale

In [2]:
import seaborn as sns
import matplotlib.pyplot as plt

In [48]:
dataset = pd.read_csv('./cleaned_dataset.csv')

In [39]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

In [49]:
X = dataset.drop(columns=['outcome'])
y = dataset.outcome

In [50]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, shuffle=True)

In [51]:
from sklearn.neural_network import MLPClassifier
mlpc = MLPClassifier(max_iter=500).fit(X_train, y_train)

In [52]:
mlpc.score(X_test, y_test)

0.7019244752532842

In [53]:
mlpc.score(X_train, y_train)

0.7065104103546065

In [54]:
y_pred = mlpc.predict(X_test)
confusion_matrix(y_test, y_pred)

array([[ 175,  844,  875,   50],
       [  13, 2304, 1473,   62],
       [  19, 1417, 8010,   23],
       [   7, 1050,  316, 3991]])

In [55]:
mlpc.predict_proba(X_test[:5]), y_test[:5]

(array([[0.00113028, 0.00161309, 0.00507632, 0.99218031],
        [0.16984093, 0.385933  , 0.27665597, 0.1675701 ],
        [0.16771102, 0.39107745, 0.27347428, 0.16773725],
        [0.00458222, 0.00176036, 0.96049258, 0.03316484],
        [0.1699404 , 0.35897571, 0.36374807, 0.10733582]]),
 28386    4
 57917    3
 58329    2
 21401    3
 89200    3
 Name: outcome, dtype: int64)

without promotion / relegation:

In [56]:
X_slim = X.drop(columns=['home_newly_relegated','away_newly_relegated','home_newly_promoted','away_newly_promoted','capacity','season','home_goals_a_cum','home_goals_f_cum','away_goals_f_cum','away_goals_a_cum'])
X_slim.columns

Index(['elo_home', 'elo_away', 'home_streak', 'away_streak'], dtype='object')

In [57]:
X_train, X_test, y_train, y_test = train_test_split(X_slim, y, test_size = 0.2, shuffle=True)
mlpc_slim = MLPClassifier(max_iter=500).fit(X_train, y_train)

In [58]:
mlpc_slim.score(X_test, y_test)

0.6332347665907218